# TPC Electron Extraction Efficiency

**Author:** Sam Eriksen

**Date:** 18 July 2023

## Description

This notebook extracts SS events and determines the ratio between the single electron pulses and S2 pulses. 
This is used to determine the extraction efficiency.

In [1]:
import utils
import copy
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [2]:
%load_ext jinja_yaml_magic
%reload_ext jinja_yaml_magic

In [3]:
%%yaml config

cluster:
    location: "nersc" # set where the dask cluster will run. options are "dirac", "nersc" and "local"
    scaling: 5 # number of dask workers. Not valid for nersc
    scheduler_options: {"port": 8786} # options for dask scheduler. Only valid for dirac

data: 
    data_location: "nersc" # set where the data will be loaded from. options are "dirac" and "nersc"
    data_path: "/lz/data/reconstructed/SR1/LZAP-5.5.1*/*/*" # April 2022 kr85m data
    event_range: [8101, 8106] # April 2022 kr85m data


## Setup dask cluster

In [ ]:
client = utils.clients.get_client(client_type=config['cluster']['location'])

## Get files

In [ ]:
files = utils.get_files.get_files(config['data']['data_location'], config['data']['data_path'], config['data']['event_range'])

## Setup and run analysis

In [ ]:
output = utils.analysis.create_output()

In [7]:
dask_tasks = utils.store.add_dask_tasks([utils.analysis.single_file_analysis(file, copy.deepcopy(output)) for file in files])

In [8]:
result = tasks.compute()

## Select S2

In [ ]:
s2_type = ['raw', 'corrected', 'xy_corrected']
bins=[np.arange(0,1000,1), np.arange(0,7,0.05)]
for s2 in s2_type:
    fig, ax = plt.subplots(1,1,figsize=(20,10))
    ax.hist2d(result['s1_phd'], np.log10(result['s2_phd_' + s2]), bins=bins, cmin=1, norm=matplotlib.colors.LogNorm())
    ax.set_ylabel(r'log$_{10}$(S2) [phd]', fontsize=20)
    ax.set_xlabel('S1 [phd]', fontsize=20)
    ax.set_title('S2 ' + s2, fontsize=30)
    plt.plot()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,10))
ax.hist(result['elife_correction'], bins=np.arange(0,2,0.01))
ax.set_ylabel('count', fontsize=20)
ax.set_xlabel(r'$\frac{{S2 corrected}}{S2 xy corrected}$', fontsize=20)
ax.set_title('Electron lifetime correction factor', fontsize=30)
plt.plot()

In [ ]:
bins=[np.arange(0,1000,1), np.arange(0,7,0.05)]
fig, ax = plt.subplots(1,1,figsize=(20,10))
ax.hist2d(result['ss_s1_phd'], np.log10(result['s2_phd_elife_corrected']), bins=bins, cmin=1, norm=matplotlib.colors.LogNorm())
ax.set_ylabel(r'log$_{10}$(S2) [phd]', fontsize=20)
ax.set_xlabel('S1 [phd]', fontsize=20)
ax.set_title('S2 electron lifetime corrected', fontsize=30)
plt.plot()

In [ ]:
s2_x_cut = [150, 400, 400, 150, 150]
s2_y_cut = [4.0, 4.0, 5.0, 5.0, 4.0]

fig, ax = plt.subplots(1,1,figsize=(20,10))
ax.hist2d(result['ss_s1_phd'], np.log10(result['s2_phd_elife_corrected']), bins=bins, cmin=1, norm=matplotlib.colors.LogNorm())
ax.plot(s2_x_cut, s2_y_cut, color='red', linewidth=3)
ax.set_ylabel(r'log$_{10}$(S2) [phd]', fontsize=20)
ax.set_xlabel('S1 [phd]', fontsize=20)
ax.set_title('S2 electron lifetime corrected', fontsize=30)
plt.plot()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,10))
ax.hist2d(result['peak_s1'], np.log10(result['peak_s2']), bins=bins, cmin=1, norm=matplotlib.colors.LogNorm())
ax.set_ylabel(r'log$_{10}$(S2) [phd]', fontsize=20)
ax.set_xlabel('S1 [phd]', fontsize=20)
ax.set_title('S2 final selection', fontsize=30)
plt.plot()

In [ ]:
bins=[np.arange(0,80,1), np.arange(1.5,7,0.05)]
fig, ax = plt.subplots(1,1,figsize=(20,10))
ax.hist2d(result['peak_s1'], np.log10(result['peak_s2']), bins=bins, cmin=1, norm=matplotlib.colors.LogNorm())
ax.set_xlabel('R', fontsize=20)
ax.set_ylabel(r'log$_{10}$(S2) [phd]', fontsize=20)
plt.plot()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20,10))
bins = np.arange(10000,50000,40)
r_bins = np.arange(0,80,5)

for r in r_bins[:-1]:
    selection = np.array(result['peak_r'] <= r+5) * np.array(result['peak_r'] >= r)
    selection_s2 = result['peak_s2'][selection]
    ax.hist(selection_s2, bins=bins, histtype='step', label='r = ' + str(r) + ' - ' + str(r+5))
    ax.set_xlabel(r'log$_{10}$(S2) [phd]', fontsize=20)
    ax.legend(fontsize=10)
    plt.plot()